In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: [Mar 24, 11:59 pm](https://github.com/UBC-CS/cpsc330-2024W2?tab=readme-ov-file#deliverable-due-dates-tentative).**

## Imports

In [2]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">
    
## Submission instructions
<hr>
rubric={points}

**Please be aware that this homework assignment requires installation of several packages in your course environment. It's possible that you'll encounter installation challenges, which might be frustrating. However, remember that solving these issues is not wasting time but it is an essential skill for anyone aspiring to work in data science or machine learning.**

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2024W2/blob/main/docs/homework_instructions.md). 

**You may work in a group on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2. 
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).


When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission.
4. Make sure that the plots and output are rendered properly in your submitted file. 
5. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb. If the pdf or html also fail to render on Gradescope, please create two files for your homework: hw6a.ipynb with Exercise 1 and hw6b.ipynb with Exercises 2 and 3 and submit these two files in your submission.  
</div>

_Points:_ 2

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words using `glove_wiki_vectors` embeddings.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [6]:
glove_wiki_vectors.most_similar('apple') 

[('microsoft', 0.7449405789375305),
 ('ibm', 0.6821643114089966),
 ('intel', 0.6778088212013245),
 ('software', 0.6775422692298889),
 ('dell', 0.6741442680358887),
 ('pc', 0.6678153276443481),
 ('macintosh', 0.66175377368927),
 ('iphone', 0.6595611572265625),
 ('ipod', 0.65346759557724),
 ('hewlett', 0.6516579985618591)]

In [7]:
glove_wiki_vectors.most_similar('sunny') 

[('chilly', 0.7509821057319641),
 ('breezy', 0.7347103953361511),
 ('windy', 0.7251654267311096),
 ('warm', 0.7054734826087952),
 ('balmy', 0.6959273219108582),
 ('cloudy', 0.670324981212616),
 ('rainy', 0.6653021574020386),
 ('sunshine', 0.6617124080657959),
 ('humid', 0.654381275177002),
 ('pleasant', 0.6506577730178833)]

In [8]:
glove_wiki_vectors.most_similar('happy') 

[("'m", 0.8413287401199341),
 ('feel', 0.8132575750350952),
 ("'re", 0.804808497428894),
 ('i', 0.7938276529312134),
 ("'ll", 0.7916273474693298),
 ('really', 0.7903971076011658),
 ('glad', 0.7833361029624939),
 ('good', 0.7821646332740784),
 ('we', 0.7808917164802551),
 ('sure', 0.7788467407226562)]

In [9]:
glove_wiki_vectors.most_similar('good') 

[('better', 0.893191397190094),
 ('sure', 0.8314563035964966),
 ('really', 0.8297762274742126),
 ('kind', 0.8288268446922302),
 ('very', 0.8260800242424011),
 ('we', 0.8234356045722961),
 ('way', 0.8215398192405701),
 ('think', 0.8205099105834961),
 ('thing', 0.8171301484107971),
 ("'re", 0.8141680955886841)]

In [10]:
glove_wiki_vectors.most_similar('our') 

[('their', 0.8240540027618408),
 ('my', 0.8232981562614441),
 ('your', 0.8203747868537903),
 ('own', 0.8164112567901611),
 ('we', 0.799706757068634),
 ('way', 0.7840150594711304),
 ('what', 0.7776114344596863),
 ('good', 0.7734648585319519),
 ('whatever', 0.7712026834487915),
 ('need', 0.7704352140426636)]

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [11]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [12]:
coast_shore_similarity = glove_wiki_vectors.similarity("coast", "shore")
clothes_closet_similarity = glove_wiki_vectors.similarity("clothes", "closet")
old_new_similarity = glove_wiki_vectors.similarity("old", "new")
smart_intelligent_similarity = glove_wiki_vectors.similarity("smart", "intelligent")
dog_cat_similarity = glove_wiki_vectors.similarity("dog", "cat")
tree_lawyer_similarity = glove_wiki_vectors.similarity("tree", "lawyer")    

coast_shore_similarity
clothes_closet_similarity
old_new_similarity
smart_intelligent_similarity
dog_cat_similarity
tree_lawyer_similarity

0.076719455

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Representation of all words in English
rubric={points}

**Your tasks:**

1. The vocabulary size of Wikipedia embeddings is quite large. The `test_words` list below contains a few new words (called neologisms) and biomedical domain-specific abbreviations. Write code to check whether `glove_wiki_vectors` have representation for these words or not. 
> If a given word `word` is in the vocabulary, `word in glove_wiki_vectors` will return True. 

In [13]:
test_words = [
    "covididiot",
    "fomo",
    "frenemies",
    "anthropause",
    "photobomb",
    "selfie",
    "pxg",  # Abbreviation for pseudoexfoliative glaucoma
    "pacg",  # Abbreviation for primary angle closure glaucoma
    "cct",  # Abbreviation for central corneal thickness
    "escc",  # Abbreviation for esophageal squamous cell carcinoma
]

<div class="alert alert-warning">

Solution_1_3
    
</div>

_Points:_ 2

In [14]:
for word in test_words:
    if word in glove_wiki_vectors:
        print(f"'{word}' is in the vocabulary.")
    else:
        print(f"'{word}' is NOT in the vocabulary.")

'covididiot' is NOT in the vocabulary.
'fomo' is NOT in the vocabulary.
'frenemies' is in the vocabulary.
'anthropause' is NOT in the vocabulary.
'photobomb' is NOT in the vocabulary.
'selfie' is NOT in the vocabulary.
'pxg' is NOT in the vocabulary.
'pacg' is NOT in the vocabulary.
'cct' is in the vocabulary.
'escc' is in the vocabulary.


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Stereotypes and biases in embeddings
rubric={points}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore this. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

In [15]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

Examples of using analogy to explore biases and stereotypes.  

In [16]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


In [17]:
glove_wiki_vectors.similarity("aboriginal", "success")

0.1428324

In [18]:
glove_wiki_vectors.similarity("white", "success")

0.351824

<div class="alert alert-warning">

Solution_1_4
    
</div>

_Points:_ 4

In [19]:
glove_wiki_vectors.similarity("black", "success")

0.37283093

In [20]:
glove_wiki_vectors.similarity("daddy", "rich")

0.23108359

In [21]:
glove_wiki_vectors.similarity("chinese", "math")

0.1911945

In [22]:
glove_wiki_vectors.similarity("asian", "chinese")

0.6650249

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Discussion
rubric={points}

**Your tasks:**
1. Discuss your observations from 1.4. Are there any worrisome biases in these embeddings trained on Wikipedia?   
2. Give an example of how using embeddings with biases could cause harm in the real world.

<div class="alert alert-warning">

Solution_1_5
    
</div>

_Points:_ 4

Yes. Black and success got higher marks than white and success. Also, the similarity between Asian and Chinese is 0.67, which is very high. I think in real life, every time people see some Asian faces, they will assume they are from China. But actually, they might be from Vietnam, Korea, or Japan. This is not respectful to those people.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The goal of topic modeling is discovering high-level themes in a large collection of texts. 

In this homework, you will explore topics in [the 20 newsgroups text dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [23]:
from sklearn.datasets import fetch_20newsgroups

In [24]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [25]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points}

Preprocessing is a crucial step before carrying out topic modeling and it markedly affects topic modeling results. In this exercise, you'll prepare the data using [spaCy](https://spacy.io/) for topic modeling. 

**Your tasks:** 

- Write code using [spaCy](https://spacy.io/) to preprocess the `text` column in the given dataframe `df` and save the processed text in a new column called `text_pp` within the same dataframe.

If you do not have spaCy in your course environment, you'll have to [install it](https://spacy.io/usage) and download the pretrained model en_core_web_md. 

`python -m spacy download en_core_web_md`


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are likely to be more informative for the given task. Some common text preprocessing steps for topic modeling include: 
- getting rid of slashes, new-line characters, or any other non-informative characters
- sentence segmentation and tokenization      
- replacing urls, email addresses, or numbers with generic tokens such as "URL",  "EMAIL", "NUM". 
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization


> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://universaldependencies.org/u/pos/) is the list of part-of-speech tags used in spaCy. 

> You may have to use regex to clean text before passing it to spaCy. Also, you might have to go back and forth between preprocessing in this exercise and and topic modeling in Exercise 2 before finalizing preprocessing steps. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you finalize the preprocessing part, you might want to save the preprocessed data in a CSV and work with this CSV so that you don't run the preprocessing part every time you run the notebook. 
 


In [26]:
import spacy
nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

In [27]:
def preprocess_spacy(
    doc,
    min_token_len=2,
    irrelevant_pos=["ADV", "PRON", "CCONJ", "PUNCT", "PART", "DET", "ADP", "SPACE"],
):
    """
    Returns the preprocessed text
    """
    clean_text = []
    
    for token in doc:
        if token.like_email:  
            clean_text.append("EMAIL")
        elif token.like_url:  
            clean_text.append("URL")
        elif token.like_num: 
            clean_text.append("NUM")
        elif (
            token.is_stop == False  
            and len(token) > min_token_len  
            and token.pos_ not in irrelevant_pos
        ): 
            clean_text.append(token.lemma_.lower())
    return " ".join(clean_text)



In [28]:
import re
def preprocess(text):
    text = re.sub(r"\s+", " ", text)
    
    text = re.sub(r"""[\n\r]+""", "", text)
    
    text = re.sub(r"""[\*\~]+""", "", text)

    text = re.sub(r"""[\/]+""", " ", text)

    return text

In [29]:
df["text_pp"] = [
    preprocess_spacy(spacy_text) for spacy_text in nlp.pipe(df["text"].tolist())
] 

In [30]:
df.iloc[2:6]

,text,target,target_name,text_pp
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have NUM newsgroup...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create NUM ...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...
5,"Hi,\n I'd like to subscribe to Leadership Ma...",5,soc.religion.christian,like subscribe leadership magazine wonder NUM ...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points}

**Your tasks:**

- Outline the preprocessing steps you carried out in the previous exercise (bullet point format is fine), providing a brief justification when necessary. 

> You might want to wait to answer this question till you are done with Exercise 2 and you have finalized the preprocessing steps in 2.1. 

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 2

I chose to apply the following preprocessing steps: First I removed characters such as slashes, newline symbols, and similar formatting artifacts. Then I performed sentence segmentation and word tokenization. Also I replaced URLs, email addresses, and numerical values with the tokens "URL", "EMAIL", and "NUM" respectively. After that, I filtered out other rare tokens that are unlikely to contribute meaningfully to topic modeling. Then, I removed common stopwords and punctuation, as they are frequent across all text and do not typically aid in identifying distinct topics. At last, I applied lemmatization to reduce words to their base forms.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points}

**Your tasks:**

1. Build LDA models on the preprocessed data using using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) and random state 42. Experiment with a few values for the number of topics (`n_components`). Pick a reasonable number for the number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 4

I selected four topics for modeling based on both domain knowledge and empirical observation. The dataset appears to broadly represent content from four main thematic areas: religion, computers, sports, and politics. These categories naturally suggest a four-topic structure. Additionally, after experimenting with different numbers of topics, I found that using four produced the most coherent and interpretable groupings of terms and documents. The topics aligned well with the known categories and provided clearer thematic separation compared to other values I tried.


In [31]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()

vec.fit(df["text_pp"])

X_vec = vec.transform(df["text_pp"])

In [32]:
from sklearn.decomposition import LatentDirichletAllocation

n_topics = 4

lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=10, random_state=42
)
document_topics = lda.fit_transform(X_vec)

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points}

**Your tasks:**
1. For the number of topics you picked in the previous exercise, show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

In [33]:
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())

In [34]:
!pip install mglearn

In [35]:
import mglearn

mglearn.tools.print_topics(
    topics=range(4),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=20,
)

topic 0       topic 1       topic 2       topic 3       
--------      --------      --------      --------      
num           num           num           num           
url           gun           game          god           
file          year          play          people        
email         people        team          know          
program       think         year          say           
use           good          season        think         
image         time          win           come          
window        know          period        believe       
available     team          armenians     time          
include       turkish       goal          thing         
entry         right         go            jesus         
server        player        get           tell          
run           armenian      armenian      way           
widget        go            new           israel        
version       say           league        question      
display       get           run

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

_Points:_ 5

In [36]:
for i in range(5):
    print('\n\nDOCUMENT', df.iloc[i]['text'])
    print("\n\nDOCUMENT TOPIC ASSIGNMENT: ", document_topics[i]) 



DOCUMENT You know, I was reading 18 U.S.C. 922 and something just did not make 
sence and I was wondering if someone could help me out.

Say U.S.C. 922 :

(1) Except as provided in paragraph (2), it shall be unlawful for
any person to transfer or possess a machinegun.

 Well I got to looking in my law dictionary and I found that a "person" 
might also be an artificial entity that is created by government 
and has no rights under the federal constitution. So, what I 
don't understand is how a statute like 922 can be enforced on 
an individual. So someone tell me how my government can tell
me what I can or cannot possess. Just passing a law 
does not make it LAW. Everyone knows that laws are constitional
until it goes to court. So, has it ever gone to court, not
just your run of the mill "Ok I had it I am guilty, put me in jail"

Has anyone ever claimed that they had a right to possess and was told
by the Supreme Court that they didn't have that right?





DOCUMENT TOPIC ASSIGNMENT:  

The model shows strong topic assignment for most documents, indicating that it is relatively confident in associating documents with one dominant topic:

	1. Document 1 has a high probability for Topic 1 (≈ 87.6%), suggesting it’s clearly related to that topic. Given the document’s focus on U.S. law and individual rights, this might indicate Topic 1 is centered around law/government/politics.
    
	2. Document 2 has high probability for Topic 0 (≈ 69.3%) and a notable presence of Topic 2 (≈ 30%). The technical nature of the content (geometry and algorithms) suggests Topic 0 may represent science/math/engineering themes, while Topic 2 might also involve related technical discussions.
    
	3.	Document 3 is strongly associated with Topic 3 (≈ 65%), pointing toward that topic as likely related to technology or computing, especially since the document discusses newsgroup structures and graphics programming.
    
	4.	Document 4 has very high confidence in Topic 0 (≈ 91%), aligning with the technical troubleshooting content (X11 window management). This reinforces that Topic 0 likely captures software/technical support discussions.
    
	5.	Document 5 is again strongly aligned with Topic 1 (≈ 76%), which supports the interpretation of Topic 1 as international affairs or politics, given the discussion of Armenia, Turkey, and humanitarian issues.

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points}

1. Briefly explain how content-based filtering works in the context of recommender systems. 
2. Discuss at least two negative consequences of recommender systems.
3. What is transfer learning in natural language processing? Briefly explain.     

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

1. Content-based filtering recommends items to users based on the features of items they have previously liked or interacted with. It builds a profile of the user using item attributes (e.g., genre, keywords, tags) and suggests new items with similar features. For example, if a user watches many romance movies, the system recommends other movies labeled as romance or with similar plots or actors.
2. One bad thing might happen is Privacy Concerns: To provide personalized recommendations, systems often collect and analyze large amounts of personal data, raising ethical and privacy issues if not handled responsibly. On the other, lack of new things and diversity: Recommender systems tend to reinforce a user’s existing preferences, limiting exposure to diverse or new content, which may lead to narrow worldviews or polarizing effects.
3.  Transfer learning in NLP refers to reusing a pre-trained language model on a new, often smaller, NLP task. The model, already trained on large text corpora, has learned general language representations and can be fine-tuned on a specific small task. This boosts performance and reduces training time.



<!-- END QUESTION -->

<br><br><br><br>

**Before submitting your assignment, please make sure you have followed all the instructions in the Submission instructions section at the top.** 

![](img/eva-well-done.png)